In [1]:
# basic imports
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import h5py
import copy


In [2]:
# define input folder and current subject. 
this_subj = '1331'
infold = '/remotedata/AgGross/TBraiC/MV-eye/STRG_computed_features/alphaSPADE/HCTSA_prestim'


In [3]:
# import all packages needed for classification
from sklearn.svm import SVC
# from sklearn.metrics import balanced_accuracy_score # not needed now, called internally from pipeline object
# functions for PCA
from sklearn.decomposition import PCA
# pipeline definer
from sklearn.pipeline import Pipeline
# scaler
from sklearn.preprocessing import RobustScaler
# for inserting tanh in pipeline
from sklearn.preprocessing import FunctionTransformer
# crossvalidation
from sklearn.model_selection import cross_val_score

# define the pipeline to be used
class_pipeline = Pipeline([('scaler', RobustScaler()),
                           ('squeezer', FunctionTransformer(np.tanh)),
                           ('std_PCA', PCA(n_components=.9, svd_solver='full')),
                           ('SVM', SVC(C=10, random_state=42))
                           ])


In [4]:
# all of the following would be the body of the dask-parallel function
# input given to the function: infold, this_subj, class_pipeline, cv_fold=5

cv_fold = 5;

# load data (h5py required for big datasets)
dat = h5py.File(infold+this_subj + '_hctsa_cat_dat.mat')['dat'] # ! '_hctsa_cat_dat.mat'
bigX = dat['X'][:]
Y = dat['Y'][0, :]
    

In [5]:
# create shuffled vector of labels
randY = copy.deepcopy(Y)
np.random.shuffle(randY)

# extract number of features computed
nfeats = bigX.shape[1]

dict_accs = {}
for ifeat in range(nfeats):

    X = bigX[:, ifeat, :]

    acc = cross_val_score(class_pipeline, X, Y, cv=cv_fold, 
                          scoring='balanced_accuracy').mean()
    rand_acc = cross_val_score(class_pipeline, X, randY, cv=cv_fold, 
                          scoring='balanced_accuracy').mean()
    dict_accs[str(ifeat)]= acc-rand_acc
    
    print(ifeat)

/home/balestrieri/PyEnvs/JupyterEnv/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:543: RuntimeWarning: divide by zero encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/home/balestrieri/PyEnvs/JupyterEnv/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:543: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/home/balestrieri/PyEnvs/JupyterEnv/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:543: RuntimeWarning: divide by zero encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/home/balestrieri/PyEnvs/JupyterEnv/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:543: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/home/balestrieri/PyEnvs/JupyterEnv/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:543: RuntimeWarning: divide by zero encountered

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


KeyboardInterrupt: 